<h1>Parking Spaces</h1>

<h2>import all libraries</h2>

In [6]:
import cv2
import pickle

from skimage.transform import resize
import numpy as np

<h2>helper functions</h2>

In [7]:
# Import necessary libraries
EMPTY = True
NOT_EMPTY = False

# Load the pre-trained machine learning model
MODEL = pickle.load(open("model.p", "rb"))

# Function to determine if a parking spot is empty or not based on a given image
def empty_or_not(spot_bgr):
    # Flatten and resize the image data
    flat_data = []
    img_resized = resize(spot_bgr, (15, 15, 3))
    flat_data.append(img_resized.flatten())
    flat_data = np.array(flat_data)

    # Predict using the pre-trained model
    y_output = MODEL.predict(flat_data)

    # Return whether the parking spot is empty or not based on the model prediction
    if y_output == 0:
        return EMPTY
    else:
        return NOT_EMPTY

# Function to extract bounding boxes of parking spots from connected components
def get_parking_spots_bboxes(connected_components):
    (totalLabels, label_ids, values, centroid) = connected_components

    # Initialize a list to store bounding boxes of parking spots
    slots = []
    coef = 1

    # Loop through connected components and extract bounding box coordinates
    for i in range(1, totalLabels):
        x1 = int(values[i, cv2.CC_STAT_LEFT] * coef)
        y1 = int(values[i, cv2.CC_STAT_TOP] * coef)
        w = int(values[i, cv2.CC_STAT_WIDTH] * coef)
        h = int(values[i, cv2.CC_STAT_HEIGHT] * coef)

        # Append bounding box coordinates to the list
        slots.append([x1, y1, w, h])

    return slots

# Function to calculate the absolute difference between the mean intensities of two images
def calc_diff(im1, im2):
    return np.abs(np.mean(im1) - np.mean(im2))


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/base.py:348: InconsistentVersionWarning: Trying to unpickle estimator SVC from version 1.1.3 when using version 1.3.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


<h2>Import the video</h2>

In [8]:
# Specify the path to the video file
video_path = './parking_1920_1080_loop.mp4'

# Open a video capture object using OpenCV
cap = cv2.VideoCapture(video_path)

<h2>find bounding boxes</h2>

In [9]:
# Specify the path to the mask image
mask_path = './mask_1920_1080.png'

# Read the mask image in grayscale
mask = cv2.imread(mask_path, 0)

# Use connected components to label and extract statistics about connected regions in the mask
connected_components = cv2.connectedComponentsWithStats(mask, 4, cv2.CV_32S)

# Get bounding boxes (x, y, width, height) for parking spots from the connected components
spots = get_parking_spots_bboxes(connected_components)

# Print the bounding box coordinates of the first parking spot
print(spots[0])

[533, 98, 65, 35]


<h2>Play and Manipulate Video</h2>

In [10]:
# Initialize lists to store parking spot statuses and differences between frames
spots_status = [None for j in spots]
diffs = [None for j in spots]

# Initialize variables for tracking the previous frame
previous_frame = None

# Initialize frame number, video capture, and processing step
frame_nmb = 0
ret = True
step = 30

# Loop through video frames
while ret:
    # Read the next frame from the video
    ret, frame = cap.read()

    # Process every 30 frames and calculate differences
    if frame_nmb % step == 0 and previous_frame is not None:
        for spot_index, spot in enumerate(spots):
            x1, y1, w, h = spot

            # Crop the current and previous frames to the parking spot
            spot_crop = frame[y1:y1 + h, x1:x1 + w, :]
            previous_spot_crop = previous_frame[y1:y1 + h, x1:x1 + w, :]

            # Calculate the difference between the current and previous frames
            diffs[spot_index] = calc_diff(spot_crop, previous_spot_crop)

    # Process every 30 frames
    if frame_nmb % step == 0:
        # Determine the order of spots to check based on differences
        if previous_frame is None:
            arr_ = range(len(spots))
        else:
            # Sort spots based on differences and filter by threshold
            arr_ = [j for j in np.argsort(diffs) if diffs[j] / np.amax(diffs) > 0.4][::-1]

        # Loop through the sorted spots
        for spot_index in arr_:
            spot = spots[spot_index]
            x1, y1, w, h = spot

            # Crop the frame to the parking spot
            spot_crop = frame[y1:y1 + h, x1:x1 + w, :]

            # Use the model to predict whether the parking spot is empty or not
            spot_status = empty_or_not(spot_crop)
            spots_status[spot_index] = spot_status

    # Process every 30 frames
    if frame_nmb % step == 0:
        # Update the previous frame
        previous_frame = frame.copy()

    # Draw rectangles and display information on the frame
    for spot_index, spot in enumerate(spots):
        spot_status = spots_status[spot_index]
        x1, y1, w, h = spots[spot_index]

        # Draw rectangles based on spot status
        if spot_status:
            cv2.rectangle(frame, (x1, y1), (x1 + w, y1 + h), (0, 255, 0), 2)
        else:
            cv2.rectangle(frame, (x1, y1), (x1 + w, y1 + h), (0, 0, 255), 2)

    # Draw a rectangle and display information about available spots
    cv2.rectangle(frame, (80, 20), (550, 80), (0, 0, 0), -1)
    cv2.putText(frame, 'Available Spots: {} / {}'.format(str(sum(spots_status)), str(len(spots_status))),
                (100, 60), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 22), 2)

    # Show the frame
    cv2.namedWindow('window', cv2.WINDOW_NORMAL)
    cv2.imshow('frame', frame)

    # Break the loop if 'q' key is pressed
    if cv2.waitKey(25) & 0xFF == ord('q'):
        break

    # Increment frame number
    frame_nmb += 1

# Release the video capture and close all windows
cap.release()
cv2.destroyAllWindows()
